# Dagworks



In [ ]:
from hamilton import driver
import my_dag

In [ ]:
dr = driver.Builder().with_modules(my_dag).build()

In [ ]:
# dr.display_all_functions("dag.png")  # outputs a file dag.png
dr.display_all_functions()

In [ ]:
dr.execute(final_vars=['get_response_from_openai'],display_graph=True)

In [ ]:
dr.visualize_execution(final_vars=['get_response_from_openai'])